To build a RAG (Retrieval-Augmented Generation) system with ChromaDB for storing and searching vectorized data, we can adapt your example to handle the `.csv` file containing Java snippets. Here's a step-by-step explanation and an example in Python:

### Steps:

1. **Load the Dataset**: Read the `javaSnippets.csv` file.
2. **Embed Text**: Use a language model to embed the Java snippets into vector representations.
3. **Store in ChromaDB**: Save the embeddings into ChromaDB for retrieval.
4. **Search for Snippets**: Perform similarity searches on the database using vectorized queries.

### 1. **Install Required Libraries**

Before starting, install the necessary libraries:

In [23]:
# pip install pandas sentence-transformers chromadb


---

### 2. **Load and Inspect Data**

In [1]:
import pandas as pd

# Step 1: Load the dataset
file_path = './datasets/snippets/javaSnippets.csv'  # Update with your file path
df = pd.read_csv(file_path)

# Inspect the first few rows of the dataset
print("Dataset Preview:")
print(df.head())

# Ensure the column with code snippets is correctly identified
column_name = 'snippet'
assert column_name in df.columns, f"Column '{column_name}' not found in the dataset."

# Extract snippets
snippets = df[column_name].tolist()
print(f"Loaded {len(snippets)} code snippets.")

Dataset Preview:
      id                                            snippet language  \
0  88608  /*\r\n * Copyright 2014 The Netty Project\r\n ...     Java   
1  88609   * with the License. You may obtain a copy of ...     Java   
2  88610   * distributed under the License is distribute...     Java   
3  88611  \r\npackage io.netty.resolver.dns;\r\n\r\nimpo...     Java   
4  88612  \r\nimport java.net.InetSocketAddress;\r\nimpo...     Java   

                                      repo_file_name  \
0  netty/netty/resolver-dns/src/test/java/io/nett...   
1  netty/netty/resolver-dns/src/test/java/io/nett...   
2  netty/netty/resolver-dns/src/test/java/io/nett...   
3  netty/netty/resolver-dns/src/test/java/io/nett...   
4  netty/netty/resolver-dns/src/test/java/io/nett...   

                  github_repo_url     license  \
0  https://github.com/netty/netty  Apache-2.0   
1  https://github.com/netty/netty  Apache-2.0   
2  https://github.com/netty/netty  Apache-2.0   
3  https://github

---
To clean your dataset (`javaSnippets.csv`), you can follow a structured process similar to the example you provided. Here's how you can clean the data:

### Step-by-Step Data Cleaning
1. **Check for Missing Values**:
   Identify which columns have missing values and decide whether to drop rows or columns or fill missing values.

2. **Remove Unnecessary Characters**:
   Since the dataset includes code snippets, it’s essential to clean characters like extra whitespace or unnecessary escape sequences (`\r\n`).

3. **Check for Duplicates**:
   Remove duplicate rows to avoid redundant data.

4. **Validate Data Types**:
   Ensure columns have correct data types (e.g., `starting_line_number` and `chunk_size` should be integers).

5. **Filter Invalid Data**:
   Remove rows with invalid or irrelevant data (e.g., empty or overly short snippets).

Here's the Python code for cleaning your dataset:

In [ ]:
# pip install dask[dataframe]

  Using cached pyarrow-18.1.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-18.1.0-cp312-cp312-win_amd64.whl (25.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
df.isna().sum()
# Columns exp and topic_name contain missing values, we will drop rows with missing values in both columns
df = df.dropna(subset=["snippet"])
df.isna().sum()
df = df.dropna(subset=["repo_file_name"])
df.isna().sum()
df = df.dropna(subset=["github_repo_url"])
df.isna().sum()

id                      0
snippet                 0
language                0
repo_file_name          0
github_repo_url         0
license                 0
commit_hash             0
starting_line_number    0
chunk_size              0
dtype: int64

### Explanation of the Code
1. **Handling Missing Values**:
   - Identifies columns with missing values and drops rows where critical columns (`snippet`, `repo_file_name`, etc.) are empty.
   
2. **Removing Unnecessary Characters**:
   - Replaces `\r\n` with `\n` for better readability of the code snippets.
   - Trims leading/trailing whitespace.

3. **Removing Duplicates**:
   - Removes rows with identical values across all columns.

4. **Validating Data Types**:
   - Ensures numeric columns like `starting_line_number` and `chunk_size` are properly formatted.

5. **Filtering Invalid Snippets**:
   - Removes rows with overly short or invalid snippets (e.g., fewer than 10 characters).

### Next Steps
- Save the cleaned dataset for further processing:
  

In [4]:
pip install datasets

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.2.1-cp312-cp312-win_amd64.whl.metadata (9.5 kB)
  Using cached yarl-1.18.3-cp312-cp312-win_amd64.whl.metadata (71 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl (30 kB)
Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl (14 kB)
Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl (51 kB)
Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl (28 kB)
Using cached propcache-0.2.1-cp312-cp312-win_amd64.whl (44 k

In [9]:
from datasets import Dataset

# Define a function to split the dataframe into smaller chunks
def process_in_chunks(df, chunk_size):
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunk = df[i * chunk_size:(i + 1) * chunk_size]
        dataset = Dataset.from_pandas(chunk)
        dataset.save_to_disk(f"./datasets/snippets/cleaned_chunk_{i}")

# Process the dataframe in chunks (adjust the chunk size as needed)
process_in_chunks(df, chunk_size=10000)  # For example, 10,000 rows per chunk


Saving the dataset (1/1 shards): 100%|██████████| 1424/1424 [00:00<00:00, 4545.83 examples/s]


In [14]:
dataset_chunk_0 = Dataset.load_from_disk("./datasets/snippets/cleaned_chunk_0")

dataset_chunk_0[0]

{'id': 88608,
 'snippet': '/*\r\n * Copyright 2014 The Netty Project\r\n *\r\n * The Netty Project licenses this file to you under the Apache License,\r\n * version 2.0 (the "License"); you may not use this file except in compliance\r\n',
 'language': 'Java',
 'repo_file_name': 'netty/netty/resolver-dns/src/test/java/io/netty/resolver/dns/DnsServerAddressesTest.java',
 'github_repo_url': 'https://github.com/netty/netty',
 'license': 'Apache-2.0',
 'commit_hash': 'a60825c3b425892af9be3e9284677aa8a58faa6b\r\n',
 'starting_line_number': 0,
 'chunk_size': 5}

---
Now that you have embeddings working fine, it's time to store those embeddings in a vector database like ChromaDB. ChromaDB is a vector database that can store embeddings efficiently and support similarity search, which is useful for tasks like semantic search.

Here’s how you can modify your code to store and retrieve embeddings using ChromaDB:

### Step 1: Install Required Libraries

Make sure you have the required libraries installed:

- `chromadb`
- `pandas`
- `numpy`

You can install ChromaDB using pip:

In [10]:
# pip install chromadb pandas numpy


---

### **Step 3: Create Collection**
You can now insert documents into the collection.

In [2]:
import chromadb
import numpy as np
import pandas as pd


# Initialize ChromaDB client
chroma_client = chromadb.Client()

# Create or connect to collection
collection_name = 'JavaCodeSnippetsDB'
if collection_name not in chroma_client.list_collections():
    chroma_client.create_collection(collection_name)

collection = chroma_client.get_collection(collection_name)


In [3]:
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
chroma_client.delete_collection(collection_name) # Deletes the collection
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------

In [3]:
if collection:
    print("ChromaDB is working! Collections found:", collection)
else:
    print("No collections found. Creating a new collection...")

ChromaDB is working! Collections found: Collection(name=JavaCodeSnippetsDB)



---

### **Step 4: Add Data to the Collection**
You can now insert documents into the collection.

In [6]:
import os
import json
from datasets import Dataset
from tqdm import tqdm


e:\Desktop\Schoole\ENSET\BDCC-2\Semester 3\POO Java\@@Project JAVA LLM\TEST\File PDF MD TEST\Ollama-Set-RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset_dir = "./datasets/snippets/"
progress_file = "./progress.json" 

# Load progress from file, if it exists
def load_progress():
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            progress = json.load(f)
        return progress
    return None

# Save progress to file
def save_progress(chunk_dir, row_idx):
    progress = {"chunk": chunk_dir, "last_row_idx": row_idx}
    with open(progress_file, 'w') as f:
        json.dump(progress, f)
    # Overwrite previous progress message
    print(f"Progress saved: {chunk_dir}, row {row_idx}", end="\r")

    
# List all the subdirectories in the directory (folders that start with 'cleaned_chunk_')
chunk_dirs = [f for f in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, f)) and f.startswith('cleaned_chunk_')]

print("Found directories:", chunk_dirs)
total_chunks = len(chunk_dirs)  # Total number of chunks
print(f"Total chunks to process: {total_chunks}")

Found directories: ['cleaned_chunk_0', 'cleaned_chunk_1', 'cleaned_chunk_10', 'cleaned_chunk_100', 'cleaned_chunk_101', 'cleaned_chunk_102', 'cleaned_chunk_103', 'cleaned_chunk_104', 'cleaned_chunk_105', 'cleaned_chunk_106', 'cleaned_chunk_107', 'cleaned_chunk_108', 'cleaned_chunk_109', 'cleaned_chunk_11', 'cleaned_chunk_110', 'cleaned_chunk_111', 'cleaned_chunk_112', 'cleaned_chunk_113', 'cleaned_chunk_114', 'cleaned_chunk_115', 'cleaned_chunk_116', 'cleaned_chunk_117', 'cleaned_chunk_118', 'cleaned_chunk_119', 'cleaned_chunk_12', 'cleaned_chunk_120', 'cleaned_chunk_121', 'cleaned_chunk_122', 'cleaned_chunk_123', 'cleaned_chunk_124', 'cleaned_chunk_125', 'cleaned_chunk_126', 'cleaned_chunk_127', 'cleaned_chunk_128', 'cleaned_chunk_129', 'cleaned_chunk_13', 'cleaned_chunk_130', 'cleaned_chunk_131', 'cleaned_chunk_132', 'cleaned_chunk_133', 'cleaned_chunk_134', 'cleaned_chunk_135', 'cleaned_chunk_136', 'cleaned_chunk_137', 'cleaned_chunk_138', 'cleaned_chunk_139', 'cleaned_chunk_14', 'c

In [9]:
# Load the last progress (if any)
progress = load_progress()

if progress:
    print(f"Resuming from chunk: {progress['chunk']}, row {progress['last_row_idx']}")
    start_chunk = progress['chunk']
    start_row_idx = progress['last_row_idx']
else:
    print("Starting from the beginning...")
    start_chunk = None
    start_row_idx = None

# Total number of chunks
total_chunks = len(chunk_dirs)
print(f"Total chunks to process: {total_chunks}")

Starting from the beginning...
Total chunks to process: 755


In [10]:
# Total number of chunks
total_chunks = len(chunk_dirs)
print(f"Total chunks to process: {total_chunks}")

Total chunks to process: 755


In [11]:
# Loop through each directory and add it to the collection
for idx, chunk_dir in enumerate(chunk_dirs, 1):
    # If the chunk is the one we need to resume, start from the last row index
    if start_chunk and chunk_dir == start_chunk:
        dataset_chunk = Dataset.load_from_disk(os.path.join(dataset_dir, chunk_dir))
        total_rows = len(dataset_chunk)
        # Process rows from the saved row index onwards
        for row_idx, first_row in enumerate(tqdm(dataset_chunk, desc=f"Processing {chunk_dir}", total=total_rows-start_row_idx, ncols=100, unit="row"), start=start_row_idx):
            # Prepare the data for adding to the collection
            ids = [str(first_row['id'])]  # Assuming 'id' is the unique identifier
            documents = [first_row['snippet']]  # Assuming 'snippet' contains the code/text to store
            metadatas = [{
                "type": "snippet",
                "repo": first_row['repo_file_name'],
                "language": first_row['language']
            }]
            
            # Add the data to the collection
            collection.add(
                ids=ids,
                documents=documents,
                metadatas=metadatas
            )
            
            # Save progress after each row
            save_progress(chunk_dir, row_idx + 1)
        
        # After completing this chunk, reset to None
        start_chunk = None  # Reset after the starting chunk is processed
        print(f"Completed {chunk_dir} chunk.\n")
    else:
        # Load the dataset chunk from the directory
        dataset_chunk = Dataset.load_from_disk(os.path.join(dataset_dir, chunk_dir))
        total_rows = len(dataset_chunk)
        
        # Iterate over each row in the dataset chunk
        for row_idx, first_row in enumerate(tqdm(dataset_chunk, desc=f"Processing {chunk_dir}", total=total_rows, ncols=100, unit="row")):
            # Prepare the data for adding to the collection
            ids = [str(first_row['id'])]  # Assuming 'id' is the unique identifier
            documents = [first_row['snippet']]  # Assuming 'snippet' contains the code/text to store
            metadatas = [{
                "type": "snippet",
                "repo": first_row['repo_file_name'],
                "language": first_row['language']
            }]
            
            # Add the data to the collection
            collection.add(
                ids=ids,
                documents=documents,
                metadatas=metadatas
            )
            
            # Save progress after each row
            save_progress(chunk_dir, row_idx + 1)
        
        # After completing this chunk, print that it's done and move to next
        print(f"Completed {chunk_dir} chunk.\n")

    # Calculate percentage of chunks processed
    chunk_progress = (idx / total_chunks) * 100
    print(f"Processed {idx}/{total_chunks} chunks ({chunk_progress:.2f}%)", end="\r")

print("\nAll chunks have been added to the collection!")

Processing cleaned_chunk_0:   0%|                              | 1/10000 [00:01<4:06:23,  1.48s/row]

Processing cleaned_chunk_0:   0%|                              | 2/10000 [00:01<2:16:31,  1.22row/s]

Processing cleaned_chunk_0:   0%|                              | 3/10000 [00:02<1:46:39,  1.56row/s]

Processing cleaned_chunk_0:   0%|                                | 5/10000 [00:02<56:32,  2.95row/s]

Processing cleaned_chunk_0:   0%|                                | 9/10000 [00:02<27:38,  6.02row/s]

...

Processing cleaned_chunk_0:  37%|██████████▋                  | 3694/10000 [14:10<44:53,  2.34row/s]

Processing cleaned_chunk_0:  37%|██████████▋                  | 3695/10000 [14:11<47:11,  2.23row/s]

Processing cleaned_chunk_0:  37%|██████████▋                  | 3696/10000 [14:11<52:04,  2.02row/s]

Processing cleaned_chunk_0:  37%|██████████▋                  | 3696/10000 [14:12<24:13,  4.34row/s]


KeyboardInterrupt: 

In [4]:
collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.embeddings: 'embeddings'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

---
### **Step 5: After the collection is ready, we can query it using the query() method of the collection and print the results returned by the query().**

In [23]:
results = collection.query(query_texts="compareTo function", n_results=3)

print(results)

{'ids': [['88884', '91245', '91029']], 'embeddings': None, 'documents': [['                    @Override\r\n                    public int compare(InetSocketAddress o1, InetSocketAddress o2) {\r\n                        if (o1.equals(o2)) {\r\n                            return 0;\r\n                        }\r\n', '\r\n    @Override\r\n    public int compare(InetAddress o1, InetAddress o2) {\r\n        if (o1.getClass() == o2.getClass()) {\r\n            return 0;\r\n', '            return Long.MIN_VALUE;\r\n        }\r\n\r\n        @Override\r\n        public int compareTo(Delayed o) {\r\n']], 'uris': None, 'data': None, 'metadatas': [[{'language': 'Java', 'repo': 'netty/netty/resolver-dns/src/test/java/io/netty/resolver/dns/DefaultAuthoritativeDnsServerCacheTest.java', 'type': 'snippet'}, {'language': 'Java', 'repo': 'netty/netty/resolver-dns/src/main/java/io/netty/resolver/dns/PreferredAddressTypeComparator.java', 'type': 'snippet'}, {'language': 'Java', 'repo': 'netty/netty/resolv

In [26]:
# # Use a query without any filters to fetch all items
# results = collection.query(query_texts="*", n_results=10000)  # Adjust the n_results as needed (limit)

# # Ensure the directory exists
# directory_path = './JavaCodeSnippetsDB'
# if not os.path.exists(directory_path):
#     os.makedirs(directory_path)

# # Define the path for the file to save the results
# file_path = os.path.join(directory_path, 'full_collection_data.json')

# # Save the entire collection data to a JSON file
# with open(file_path, 'w') as f:
#     json.dump(results, f)

# print(f"Entire collection data saved to {file_path}")

Number of requested results 10000 is greater than number of elements in index 3696, updating n_results = 3696


Entire collection data saved to ./JavaCodeSnippetsDB\full_collection_data.json
